In [1]:
import os
os.chdir('/home/victorhuang/projects/gtx/')

import numpy as np
import torch

from kan import *
from code_kan.kan_preprocess import load_data


In [2]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Number of GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"\nGPU {i}:")
        print("  Name:", torch.cuda.get_device_name(i))
        print("  Memory Allocated:", round(torch.cuda.memory_allocated(i) / 1024**2, 2), "MB")
        print("  Memory Cached:", round(torch.cuda.memory_reserved(i) / 1024**2, 2), "MB")
        print("  Capability:", torch.cuda.get_device_capability(i))
        print("  Current Device:", torch.cuda.current_device())
else:
    print("No GPU available.")

PyTorch version: 2.2.2+cu121
CUDA available: True
Number of GPUs: 1

GPU 0:
  Name: NVIDIA GeForce RTX 4070 Laptop GPU
  Memory Allocated: 0.0 MB
  Memory Cached: 0.0 MB
  Capability: (8, 9)
  Current Device: 0


# load data

In [3]:
dt_data_path = 'data/20241118_data_splited.mat'
data = load_data(dt_data_path, data_type='dt')

train_data = data['train']
train_fluorescence = train_data['fluorescence']
train_op = train_data['op']
train_depth = train_data['depth']
train_concentration_fluor = train_data['concentration_fluor']
train_reflectance = train_data['reflectance']

val_data = data['val']
val_fluorescence = val_data['fluorescence']
val_op = val_data['op']
val_depth = val_data['depth']
val_concentration_fluor = val_data['concentration_fluor']
val_reflectance = val_data['reflectance']

test_data = data['test']
test_fluorescence = test_data['fluorescence']
test_op = test_data['op']
test_depth = test_data['depth']
test_concentration_fluor = test_data['concentration_fluor']
test_reflectance = test_data['reflectance']


# KAN